<a href="https://colab.research.google.com/github/krahul2024/machine-learning/blob/main/LightGCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Better Recommender systems with LightGCN

In this tutorial, we will implement the LightGCN layer and corresponding graph neural network in order to improve recommendation performance over Matrix Factorization methods.

You will need a GPU accelarator for this. Please click Runtime and then Change runtime type. Then set the hardware accelerator to GPU.

## Data loading and preprocessing

In [1]:
import pandas as pd

We will pull the data and other helper scripts from our GitHub repository:

In [2]:
# Clone repository from git
!git clone --branch develop https://github.com/tm1897/mlg_cs224w_project.git

# Add it to system path
import sys
sys.path.append("mlg_cs224w_project")

# Import the packages
from src.data_preprocessing import TrainTestGenerator
from src.evaluator import Evaluator

Cloning into 'mlg_cs224w_project'...
remote: Enumerating objects: 243, done.
remote: Counting objects: 100% (243/243), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 243 (delta 116), reused 192 (delta 78), pack-reused 0
Receiving objects: 100% (243/243), 5.02 MiB | 6.71 MiB/s, done.
Resolving deltas: 100% (116/116), done.


TrainTestGenerator will yield train and test data points for each year (2008-2010) following the forward chaining technique.

In [3]:
data_dir = "mlg_cs224w_project/data/"
data_generator = TrainTestGenerator(data_dir)

data_generator.prepare_data()

,userID,artistID,weight,tagID,timestamp
0,2,52,11690,13,2009-03-31 22:00:00
1,2,63,3735,13,2009-03-31 22:00:00
2,2,73,2584,13,2009-03-31 22:00:00
3,2,94,1373,13,2009-03-31 22:00:00
4,2,96,1342,19,2009-03-31 22:00:00
...,...,...,...,...,...
20659,2100,1111,1062,574,2009-02-28 23:00:00
20660,2100,6658,731,4,2009-09-30 22:00:00
20661,2100,8322,650,4,2009-04-30 22:00:00
20662,2100,13978,535,574,2009-05-31 22:00:00


# Matrix Factorization - Baseline model
Matrix Factorization is often used as a baseline model, because it is simple and robust. It will give as an idea of what can be achieved with a simple model, and provide a line that has to be improved by more complex models, in order for it to be useful.


In [4]:
# Install and import the matrix factorization package (cmfrec).
!pip install cmfrec
from cmfrec import CMF_implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached findblas-0.1.21-py3-none-any.whl
  Created wheel for cmfrec: filename=cmfrec-3.5.1.post7-cp310-cp310-linux_x86_64.whl size=5830762 sha256=4ac9756dea80a91efcc364e46ba462236a1e49f1cd0855435a5f220840ceffae
  Stored in directory: /root/.cache/pip/wheels/10/cf/c1/3b05df5e49ce0c5ed2e8fe63f377c6c2c23a63c8b327d8f734
Successfully built cmfrec


We will use **CMF_implicit** model from *cmfrec* library as our matrix factorization model. We will also create a wrapper for tidy evaluation pipeline.

In [5]:
# Model wrapper
# Required by our evaluation framework

class CMF_recommender:
    def __init__(self, k=50):
        self.model = CMF_implicit(
            k=k,
            random_state=1,
        )

    def fit(self, data: pd.DataFrame):
        data = data.copy()
        data["weight"] = 1  # Binary adjacency matrix (no weights)
        data = data.rename(columns={
            "userID": "UserId",
            "artistID": "ItemId",
            "weight": "Rating"
        })
        self.model.fit(data)

    def recommend(self, user_id, n):
        recommendations = self.model.topN(user_id, n=n)
        return recommendations

Our Evaluator class accepts a model and a data generator (of type TrainTestGenerator). It will train a model on every training set provided by TrainTestGenerator and evaluate it.

In [6]:
# Evaluator (forward chaining)

evaluator = Evaluator(CMF_recommender, data_generator)
evaluator.evaluate()

We can then calculate the evaluation metrics:

In [7]:
# Hit Rate

evaluator.get_recalls()

,cases,5,10,25,50,500
2008,4556,0.015364,0.025461,0.044996,0.057946,0.231124
2009,4687,0.018562,0.028803,0.053552,0.077662,0.234905
2010,6133,0.020871,0.036198,0.067993,0.104680,0.268873


In [8]:
# Mean Reciprocal Rank

evaluator.get_mrr()

,cases,mrr
2008,2608,0.022975
2009,3086,0.024215
2010,4306,0.027039


Additionally, we can calculate the training and recommendation times, and use them later for comparison:

In [9]:
# Times

evaluator.get_times()

,count,mean,std,min,25%,50%,75%,max
task,,,,,,,,
model_fit,3.0,0.576890,0.385220,0.210981,0.375894,0.540806,0.759844,0.978882
model_init,3.0,0.000040,0.000015,0.000024,0.000033,0.000041,0.000047,0.000053
recommend_user,2336.0,0.004465,0.004377,0.000945,0.001205,0.002871,0.006143,0.041221


In [10]:
evaluator.get_fit_per_year_times()

,tag,time
task,,
model_fit,model_fit_2008,0.210981
model_fit,model_fit_2009,0.540806
model_fit,model_fit_2010,0.978882


# LightGCN

We will use PyG for the implementation of LightGCN layers and the corresponding Graph Neural Network. PyG (PyTorch Geometric) is a library built upon PyTorch to easily write and train Graph Neural Networks (GNNs) for a wide range of applications related to structured data. Documentation can be found [here](https://pytorch-geometric.readthedocs.io/en/latest/).

In [15]:
!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 45.8 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 36.9 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.3 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-8dx5sb8j
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-8dx5sb8j
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit de8f5615b789effb357c687b087c233eb96faec5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.4.0-py3-none-any.whl size=10

In [16]:
import torch
torch.manual_seed(42)

## Data loading and BipartiteData class




 After downloading and preprocessing the data, it is time to load it into PyTorch Geometric's Data object. However, since we are dealing with pure bipartite graph data, it would be the best if we customized the Data object to better fit our needs. That's why we will we write the BipartiteData class which extends the Data class, with a few additional parameters.



In [17]:
from torch_geometric.data import Data

class BipartiteData(Data):
    def __init__(self, edge_index_u2a=None, edge_index_a2u=None, num_artists=None, num_users=None):
        super().__init__()
        self.edge_index_u2a = edge_index_u2a
        self.edge_index_a2u = edge_index_a2u
        self.num_users = num_users
        self.num_artists = num_artists

    def __inc__(self, key, value, *args, **kwargs):
        # Returns the incremental count to cumulatively increase the value
        # of the next attribute of :obj:`key` when creating batches.
        if key == 'edge_index_u2a':
            return torch.tensor([[self.num_users], [self.num_artists]])
        elif key == 'edge_index_a2u':
            return torch.tensor([[self.num_artists], [self.num_users]])
        else:
            return super(BipartiteData, self).__inc__(key, value)

We will also have a function which creates this object from data files:

In [18]:
from src.get_pyg_data import load_bipartitedata

## LightGCN Layer

LightGCN follows simple aggregation and update functions:

\begin{align}
  e_{u}^{(k+1)} = \sum_{i \in N_{u}} \frac{1}{\sqrt{|N_{u}|}\sqrt{|N_{i}|}}e_{i}^{(k)} \\
  e_{i}^{(k+1)} = \sum_{i \in N_{i}} \frac{1}{\sqrt{|N_{i}|}\sqrt{|N_{u}|}}e_{u}^{(k)}
\end{align}

So it uses mean aggregation, and the message function is just the embedding from previous layer. We aggregate only the connected neighbors and do not integrate the target node itself.

Using the MessagePassing module, this can be easily implemented. We need to define *forward*, *message* and *aggregate* functions:

In [19]:
import torch.nn as nn
import torch_scatter
from torch_geometric.nn.conv import MessagePassing

In [20]:
class LightGCN(MessagePassing):
    def __init__(self, **kwargs):
        super(LightGCN, self).__init__(node_dim=0, **kwargs)

    def forward(self, x, edge_index, size=None):
        return self.propagate(edge_index=edge_index, x=(x[0], x[1]), size=size)

    def message(self, x_j):
        return x_j

    def aggregate(self, inputs, index, dim_size=None):
        return torch_scatter.scatter(src=inputs, index=index, dim=0, dim_size=dim_size, reduce='mean')

## LightGCNStack class

Now we need to create the whole graph neural network module that uses LightGCN layers. We are going to extend the *torch.nn.Module* class, so we will need to define the *forward* function, which will represent a forward pass through the network.

The main parameters of the network, that will be updated through training, are first-layer embeddings. To ease the manipulation, we will have one variable representing the user embeddings and one for artist embeddings, and we can keep them in *torch.nn.Embedding* objects.

The final embedding is calculated as the weighted sum of the embeddings of all layers:

\begin{align}
  e_{u}=\sum_{k=0}^{K} \alpha_{k}e_{u}^{(k)}, \\
  e_{i}=\sum_{k=0}^{K} \alpha_{k}e_{u}^{(i)},
\end{align}

where the weights $\alpha_{k}$ are initialized as $\alpha_{k}=\frac{1}{N+1}$, where N is the number of layers.

The final ranking score between a user and an artist is then just a dot product between their respected embeddings:

\begin{align}
  y_{ui} = e_{u}^{T}e_{i}
\end{align}

We will also need to implement the loss function for training - Bayesian Personalized Ranking loss, defined as:

\begin{align}
  L_{BPR} = - \sum_{u=1}^{M}\sum_{i \in N_{u}}\sum_{j \notin N_{u}}ln \sigma(y_{ui} - y_{uj}) + \lambda||E^{(0)}||^{2}
\end{align}



In [21]:
class LightGCNStack(torch.nn.Module):
    def __init__(self, args):
        super(LightGCNStack, self).__init__()
        self.latent_dim = args.latent_dim
        self.num_layers = args.num_layers
        self.dataset = None
        self.embeddings_users = None
        self.embeddings_artists = None
        self.lambda_reg = args.lambda_reg

        conv_model = LightGCN
        self.convs = nn.ModuleList()
        self.convs.append(conv_model())
        assert (args.num_layers >= 1), 'Number of layers is not >=1'
        for l in range(args.num_layers-1):
            self.convs.append(conv_model())

    def reset_parameters(self):
        self.embeddings.reset_parameters()

    def init_data(self, dataset):
        self.dataset = dataset
        self.embeddings_users = torch.nn.Embedding(num_embeddings=dataset.num_users, embedding_dim=self.latent_dim).to('cuda')
        self.embeddings_artists = torch.nn.Embedding(num_embeddings=dataset.num_artists, embedding_dim=self.latent_dim).to('cuda')

    def forward(self):
        x_users, x_artists = self.embeddings_users.weight, self.embeddings_artists.weight

        final_embeddings_users = torch.zeros(size=x_users.size(), device='cuda')
        final_embeddings_artists = torch.zeros(size=x_artists.size(), device='cuda')
        final_embeddings_users = final_embeddings_users + x_users/(self.num_layers + 1)
        final_embeddings_artists = final_embeddings_artists + x_artists/(self.num_layers+1)
        for i in range(self.num_layers):
            x_users = self.convs[i]((x_artists, x_users), self.dataset.edge_index_a2u, size=(self.dataset.num_artists, self.dataset.num_users))
            x_artists = self.convs[i]((x_users, x_artists), self.dataset.edge_index_u2a, size=(self.dataset.num_users, self.dataset.num_artists))
            final_embeddings_users = final_embeddings_users + x_users/(self.num_layers+1)
            final_embeddings_artists = final_embeddings_artists + x_artists/(self.num_layers + 1)

        return final_embeddings_users, final_embeddings_artists


    def decode(self, z1, z2, pos_edge_index, neg_edge_index):
        '''
        Getting recommendation scores for the edges in pos_edge_index and neg_edge_index.
        z1 and z2 are torch.nn.Embeddings objects. If edge index is of form
        (user, artist) then z1 will be user embedding matrix and z2 will be
        artist embedding matrix, else the parameters are flipped.
        '''
        edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1)  # concatenate pos and neg edges
        logits = (z1[edge_index[0]] * z2[edge_index[1]]).sum(dim=-1)  # dot product
        return logits

    def decode_all(self, z_users, z_artists):
        '''
        Get ranking score matrix for all combinations of users and artists
        '''
        prob_adj = z_users @ z_artists.t() # dot product between all combinations
        return prob_adj

    def BPRLoss(self, prob_adj, real_adj, edge_index):
        '''
        Custom written BPR Loss function. It uses full-batch calculation, so it
        requires a lot of resources and does not scale for very large graphs.
        For our dataset, it will do.

        prob_adj: NxM ranking score matrix for all users and artists
        real_adj: Real adjacency matrix of type scipy.sparse.coo_matrix
        edge_index: index of graph edges
        '''
        loss = 0
        pos_scores = prob_adj[edge_index.cpu().numpy()]
        for pos_score, node_index in zip(pos_scores, edge_index[0]):
            neg_scores = prob_adj[node_index, real_adj[node_index] == 0]
            loss = loss - torch.sum(torch.log(torch.sigmoid(pos_score.repeat(neg_scores.size()[0]) - neg_scores))) / \
                   neg_scores.size()[0]

        loss += self.lambda_reg*(torch.pow(torch.norm(self.embeddings_users.weight, dim=None), 2) +
                                 torch.pow(torch.norm(self.embeddings_artists.weight), 2))

        return loss

    def topN(self, user_id, n):
        '''
        Get indices of top N recommendations for user with ID user_id based on
        ranking scores.
        '''
        z_users, z_artists = self.forward()
        scores = torch.squeeze(z_users[user_id] @ z_artists.t())
        return torch.topk(scores, k=n)

Now we can write our training function:

In [22]:
import scipy
from torch_geometric.utils import negative_sampling

def to_scipy_sparse_matrix(edge_index, num_nodes):
    row, col = edge_index.cpu()
    edge_attr = torch.ones(row.size(0))
    out = scipy.sparse.coo_matrix(
        (edge_attr.numpy(), (row.numpy(), col.numpy())), (num_nodes[0], num_nodes[1]))
    return out

def train(model, data, optimizer):
    model.train()
    data.neg_edge_index_u2a = negative_sampling(
        edge_index=data.edge_index_u2a,  # positive edges
        num_nodes=(data.num_users, data.num_artists),  # number of nodes
        num_neg_samples=data.edge_index_u2a.size(1),
        method='sparse').to('cuda')  # number of neg_sample equal to number of pos_edges

    optimizer.zero_grad()

    z_users, z_artists = model.forward()  # encode
    loss = model.BPRLoss(model.decode_all(z_users, z_artists),
                         to_scipy_sparse_matrix(data.edge_index_u2a, num_nodes=(data.num_users, data.num_artists)).toarray(),
                         data.edge_index_u2a)

    loss.backward()
    optimizer.step()

    return loss

We will also need a tiny class for dictionary wrapping:

In [23]:
class objectview(object):
    def __init__(self, *args, **kwargs):
        d = dict(*args, **kwargs)
        self.__dict__ = d

Now we can wrap our GNN model into wrapper for tidy evaluation pipeline:

In [24]:
import time

# Wrapper for evaluation
class LightGCN_recommender:
    def __init__(self, args):
        self.args = objectview(args)
        self.model = LightGCNStack(args=self.args).to('cuda')
        self.a_rev_dict = None
        self.u_rev_dict = None
        self.a_dict = None
        self.u_dict = None

    def fit(self, data: pd.DataFrame):
        # Default rankings when userID is not in training set
        self.default_recommendation = data["artistID"].value_counts().index.tolist()

        # LightGCN
        data, self.u_rev_dict, self.a_rev_dict, self.u_dict, self.a_dict = load_bipartitedata(data)
        data = data.to("cuda")
        self.model.init_data(data)
        self.optimizer = torch.optim.Adam(params=self.model.parameters(), lr=0.001)

        best_val_perf = test_perf = 0

        for epoch in range(1, self.args.epochs+1):
            start = time.time()
            train_loss = train(self.model, data, self.optimizer)
            log = 'Epoch: {:03d}, Loss: {:.4f}, Elapsed time: {:.2f}'
            print(log.format(epoch, train_loss, time.time()-start))

    def recommend(self, user_id, n):
        try:
            recommendations = self.model.topN(self.u_dict[str(user_id)], n=n)
        except KeyError:

            recommendations = self.default_recommendation
        else:
            recommendations = recommendations.indices.cpu().tolist()
            recommendations = list(map(lambda x: self.a_rev_dict[x], recommendations))
        return recommendations

That is all! We can now run the same code fragment as in the Matrix Factorization case, with a LightGCN wrapper and LightGCN arguments.

Warning: The training takes a long time (approx. 3 hours).

In [26]:
from functools import partial

args = {'model_type': 'LightGCN', 'num_layers': 3, 'latent_dim': 32,
         'dropout': 0, 'epochs': 10, 'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0, 'weight_decay': 5e-3,
         'lr': 0.1, 'lambda_reg': 1e-4}

evaluator = Evaluator(partial(LightGCN_recommender, args), data_generator)
evaluator.evaluate()

Epoch: 001, Loss: 2062.4807, Elapsed time: 3.96
Epoch: 002, Loss: 2056.7681, Elapsed time: 2.70
Epoch: 003, Loss: 2051.0557, Elapsed time: 2.44
Epoch: 004, Loss: 2045.3394, Elapsed time: 2.45
Epoch: 005, Loss: 2039.6206, Elapsed time: 2.43
Epoch: 006, Loss: 2033.8951, Elapsed time: 3.05
Epoch: 007, Loss: 2028.1703, Elapsed time: 2.96
Epoch: 008, Loss: 2022.4492, Elapsed time: 2.47
Epoch: 009, Loss: 2016.7228, Elapsed time: 2.44
Epoch: 010, Loss: 2010.9933, Elapsed time: 2.39
Epoch: 001, Loss: 4541.3525, Elapsed time: 6.12
Epoch: 002, Loss: 4530.7808, Elapsed time: 6.50
Epoch: 003, Loss: 4520.1812, Elapsed time: 6.06
Epoch: 004, Loss: 4509.5928, Elapsed time: 6.68
Epoch: 005, Loss: 4498.9829, Elapsed time: 6.30
Epoch: 006, Loss: 4488.3755, Elapsed time: 6.40
Epoch: 007, Loss: 4477.7573, Elapsed time: 6.09
Epoch: 008, Loss: 4467.1133, Elapsed time: 6.46
Epoch: 009, Loss: 4456.4785, Elapsed time: 6.11
Epoch: 010, Loss: 4445.8232, Elapsed time: 6.70
Epoch: 001, Loss: 7276.9766, Elapsed tim

Alternatively, we can load the results of the pretrained model:

In [27]:
evaluator.load_results("./mlg_cs224w_project/results/lightgcn.csv", "./mlg_cs224w_project/results/lightgcn_time.csv")

When the evaluator finishes training and evaluating, we can print the results:

In [28]:
evaluator.get_recalls()

,cases,5,10,25,50,500
2008,4556,0.027875,0.046971,0.079675,0.133231,0.386743
2009,4687,0.032430,0.053126,0.101984,0.155323,0.433113
2010,6133,0.031958,0.065058,0.113974,0.167129,0.455079


In [29]:
evaluator.get_mrr()

,cases,mrr
2008,2608,0.039041
2009,3086,0.038456
2010,4306,0.041523


In [30]:
evaluator.get_times()

,count,mean,std,min,25%,50%,75%,max
task,,,,,,,,
model_fit,3.0,4335.942015,3190.831504,1528.726838,2600.810720,3672.894603,5739.549604,7806.204606
model_init,3.0,0.000518,0.000049,0.000464,0.000497,0.000531,0.000545,0.000559
recommend_user,2336.0,0.000768,0.000877,0.000009,0.000014,0.000019,0.001698,0.003087


In [31]:
evaluator.get_fit_per_year_times()

,tag,time
task,,
model_fit,model_fit_2008,1528.726838
model_fit,model_fit_2009,3672.894603
model_fit,model_fit_2010,7806.204606
